# Transformer Encoder from scratch using PyTorch
## 목표: self attention을 직접 구현하여 입력 문장의 문맥을 포함하는 벡터 표현 만들기

In [18]:
import torch
import torch.nn as nn
import math

In [12]:
input_text = "My name is Beomung Jeong. I really love computer vision."
tokens=input_text.replace('.','').split(' ')
vocab = {idx:token for idx, token in enumerate(tokens)}
print(vocab)

encoder_input=torch.tensor([[0,1,2,3,4],[5,6,7,8,9]])
encoder_input

{0: 'My', 1: 'name', 2: 'is', 3: 'Beomung', 4: 'Jeong', 5: 'I', 6: 'really', 7: 'love', 8: 'computer', 9: 'vision'}


tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

## 토큰 임베딩 & 위치 인코딩

In [3]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embed_dim = embed_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim)
    def forward(self,x):
        return (self.embed_dim**0.5) * self.embedding(x)   # 토큰 임베딩이 포지셔널 인코딩에 비하여 값이 너무 작지 않도록 스케일링.

class PositionalEncoding(nn.Module):
    def __init__(self, len_max_seq , embed_dim):
        super().__init__()
        pe = torch.zeros(len_max_seq ,embed_dim)
        for pos in range(len_max_seq):
            for i in range(embed_dim//2):
                pe[pos,2*i]   = math.sin( pos / (10000**((2*i)/embed_dim)) )
                pe[pos,2*i+1] = math.cos( pos / (10000**((2*i)/embed_dim)) )
        pe = pe.unsqueeze(0)            # 배치 차원 추가
        self.register_buffer('pe', pe)  # 포지셔널 인코딩은 학습하지 않으므로 레지스터 버퍼로 등록한다. 부모 클래스(nn.Module)의 메서드.
        
    def forward(self, len_seq):
        return self.pe[:,:len_seq,:]    

## 멀티 헤드 어텐션
클래스 하나에 모든 헤드의 셀프 어텐션을 한 번에 구현하였다.

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim  = embed_dim//num_heads
        self.w_q = nn.Linear(embed_dim, embed_dim)
        self.w_k = nn.Linear(embed_dim, embed_dim)
        self.w_v = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(embed_dim,embed_dim)
        
    def scaled_dot_product_attention(self, Q, K, V):
        d_k = Q.size(-1)
        return torch.matmul( torch.softmax( torch.matmul(Q,K.transpose(-2,-1))/d_k**0.5 , dim=-1 ) , V )

    def forward(self, x):
        batch_size = x.shape[0]
        len_seq    = x.shape[1]
        # transpose(1,2): 쿼리, 키, 밸류의 마지막 두 차원이 (시퀀스 길이 x 헤드 하나의 차원)이 되도록 사용.
        Q = self.w_q(x).view(batch_size, len_seq, self.num_heads, self.head_dim).transpose(1,2)
        K = self.w_k(x).view(batch_size, len_seq, self.num_heads, self.head_dim).transpose(1,2) 
        V = self.w_v(x).view(batch_size, len_seq, self.num_heads, self.head_dim).transpose(1,2)
        x = self.scaled_dot_product_attention(Q,K,V)
        x = self.dropout(x)
        x = x.contiguous().view(batch_size,len_seq,self.embed_dim)
        x = self.fc(x)
        return x

## 레이어 정규화
임베딩차원의 방향으로 레이어 정규화를 진행하는 클래스이다.
torch의 브로드캐스팅 기능을 사용하는 대신 직접 텐서의 차원을 맞추는 과정을 포함하였다.

In [5]:
class LayerNorm(nn.Module):
    def __init__(self,embed_dim):
        super().__init__()
        self.eps=1e-5
        self.gamma = nn.Parameter(torch.ones(embed_dim))        # 스케일링 인자 초기화
        self.beta  = nn.Parameter(torch.zeros(embed_dim))       # 시프팅 인자 초기화
        
    def forward(self, x):
        batch_size = x.shape[0]
        len_seq    = x.shape[1]
        embed_dim  = x.shape[2]
        
        # expand를 사용하여 텐서의 차원을 동일하게 변경
        mean  = x.mean(dim=-1,keepdim=True).expand(batch_size, len_seq, embed_dim)
        var   = x.var(dim=-1,keepdim=True).expand(batch_size, len_seq, embed_dim)
        gamma = self.gamma.expand(batch_size, len_seq, embed_dim)
        beta  = self.beta.expand(batch_size, len_seq, embed_dim)
        
        return gamma * ((x-mean)/(torch.sqrt(var)+self.eps)) + beta

## 피드 포워드

In [6]:
class FeedForward(nn.Module):
    def __init__(self, embed_dim, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, embed_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        
    def forward(self,x):
        x=self.fc1(x)
        x=self.relu(x)
        x=self.dropout(x)
        x=self.fc2(x)
        return x

## 인코더 레이어

In [7]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_heads):
        super().__init__()
        self.ffn = FeedForward(embed_dim, hidden_dim)
        self.mha = MultiHeadAttention(embed_dim, num_heads)
        self.ln1 = LayerNorm(embed_dim)
        self.ln2 = LayerNorm(embed_dim)
        self.do1 = nn.Dropout(0.1)
        self.do2 = nn.Dropout(0.1)
        
    def forward(self, x):
        x = self.ln1(x + self.do1(self.mha(x)))
        x = self.ln2(x + self.do2(self.ffn(x)))
        return x

## 인코더

In [8]:
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, len_max_seq, embed_dim, hidden_dim, num_heads, num_layers):
        super().__init__()
        self.token_embedding     = TokenEmbedding(vocab_size, embed_dim)
        self.positional_encoding = PositionalEncoding(len_max_seq, embed_dim)
        self.dropout = nn.Dropout(0.1)
        self.layers = nn.ModuleList([TransformerEncoderLayer(embed_dim, hidden_dim, num_heads) for _ in range(num_layers)])
    def forward(self, x):
        batch_size = x.shape[0]
        len_seq    = x.shape[1]
        x = self.token_embedding(x) + self.positional_encoding(len_seq)
        x = self.dropout(x)
        for layer in self.layers:
            x = layer(x)
        return x

## 인코더 객체 만들기

In [15]:
transformer_encoder=TransformerEncoder(vocab_size  = len(vocab),
                                       len_max_seq = 5,
                                       embed_dim   = 128,
                                       hidden_dim  = 512,
                                       num_heads   = 2,
                                       num_layers  = 5
                                      )

## 인코더 입력 & 인코더 출력 텐서 모양 확인 

In [16]:
encoder_output=transformer_encoder(encoder_input)
print(encoder_input.shape)
print(encoder_output.shape)

torch.Size([2, 5])
torch.Size([2, 5, 128])


## 인코더 출력 결과 확인

In [17]:
encoder_output

tensor([[[-0.9442,  1.1392,  0.1584,  ..., -0.9235,  0.3670, -0.7178],
         [-0.5536,  0.1302, -0.9800,  ..., -0.5950,  0.9755, -0.4821],
         [ 0.2435, -0.7550, -0.5374,  ..., -1.3884, -0.7751,  0.0983],
         [ 0.5625,  0.5783, -0.3128,  ..., -0.7499, -0.9429, -0.7940],
         [-1.4342,  0.4418,  0.1311,  ..., -0.1247,  0.5048, -0.3715]],

        [[-1.1368,  1.3688,  2.2270,  ...,  0.1017, -0.6633, -0.0391],
         [ 1.1998,  1.4955,  1.3791,  ...,  0.4756,  0.5808, -0.4179],
         [-0.9269,  0.6578,  0.9291,  ..., -0.3846,  0.3989,  0.0327],
         [-0.4421,  0.8831, -0.2456,  ..., -1.2086, -0.3106, -0.4489],
         [-0.8949, -0.1022,  1.7923,  ...,  0.5330, -0.6286, -0.8362]]],
       grad_fn=<AddBackward0>)